<a href="https://colab.research.google.com/github/Reuben17/Data_cleaning/blob/main/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Obtaining the Dataset

In [ ]:
import pandas as pd
import io
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LassoCV

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving modified_data.csv to modified_data.csv


In [ ]:
dataset = pd.read_csv(io.BytesIO(uploaded['modified_data.csv']))
print(dataset)

        Id  MSSubClass MSZoning  LotFrontage  LotArea Street LotShape  \
0        1          60       RL         65.0     8450   Pave      Reg   
1        2          20       RL         80.0     9600   Pave      Reg   
2        3          60       RL         68.0    11250   Pave      IR1   
3        4          70       RL         60.0     9550   Pave      IR1   
4        5          60       RL         84.0    14260   Pave      IR1   
...    ...         ...      ...          ...      ...    ...      ...   
1455  1456          60       RL         62.0     7917   Pave      Reg   
1456  1457          20       RL         85.0    13175   Pave      Reg   
1457  1458          70       RL         66.0     9042   Pave      Reg   
1458  1459          20       RL         68.0     9717   Pave      Reg   
1459  1460          20       RL         75.0     9937   Pave      Reg   

     LandContour Utilities LotConfig  ... EnclosedPorch 3SsnPorch ScreenPorch  \
0            Lvl    AllPub    Inside  ... 

# Dataset Details and Dropping

In [ ]:
dataset.head(2)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,5,2007,WD,Normal,181500


In [ ]:
dataset.shape

(1460, 77)

In [ ]:
dataset.drop('Id',inplace = True,axis=1)

In [ ]:
dataset.head(2)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500


In [ ]:
original_features=list(dataset.columns)
len(original_features)

76

It is zero as now everything has been filled

In [ ]:
features_missing_values=list(dataset.columns[dataset.isna().any()])
len(features_missing_values)

15

In [ ]:
len(dataset)

1460

# Filling Missing Features

In [ ]:
bfill,mean_fill=[],[]

In [ ]:
for feature in features_missing_values:
  print(feature,' ',dataset[feature].isna().sum()/len(dataset)*100,' ',dataset[feature].dtype)
  if dataset[feature].dtype=='float64':
    mean_fill.append(feature)
  else:
    bfill.append(feature)

LotFrontage   17.73972602739726   float64
MasVnrType   0.547945205479452   object
MasVnrArea   0.547945205479452   float64
BsmtQual   2.5342465753424657   object
BsmtCond   2.5342465753424657   object
BsmtExposure   2.6027397260273974   object
BsmtFinType1   2.5342465753424657   object
BsmtFinType2   2.6027397260273974   object
Electrical   0.0684931506849315   object
FireplaceQu   47.26027397260274   object
GarageType   5.5479452054794525   object
GarageYrBlt   5.5479452054794525   float64
GarageFinish   5.5479452054794525   object
GarageQual   5.5479452054794525   object
GarageCond   5.5479452054794525   object


In [ ]:
mean_fill

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [ ]:
for feature in mean_fill:
  dataset[feature].fillna(dataset[feature].mean(),inplace=True)

In [ ]:
bfill

['MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond']

In [ ]:
for feature in bfill:
  dataset[feature].fillna(method='bfill',inplace=True)

In [ ]:
dataset.columns[dataset.isna().any()]

Index(['FireplaceQu'], dtype='object')

In [ ]:
dataset['FireplaceQu'].isna().sum()

2

In [ ]:
dataset['FireplaceQu'].fillna(method='ffill',inplace=True)

In [ ]:
dataset.head(1)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500


# Encoding Dataset

In [ ]:
categorical_features=list(dataset.select_dtypes(include=['object']).columns)
categorical_features

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [ ]:
numerical_features=list(set(original_features)-set(categorical_features))
numerical_features

['HalfBath',
 'Fireplaces',
 'WoodDeckSF',
 '2ndFlrSF',
 'YearRemodAdd',
 'LotFrontage',
 'PoolArea',
 'MiscVal',
 'BsmtFinSF1',
 'MoSold',
 'FullBath',
 'LowQualFinSF',
 'BsmtUnfSF',
 'TotRmsAbvGrd',
 '3SsnPorch',
 'YrSold',
 'BsmtFullBath',
 'SalePrice',
 'MasVnrArea',
 'YearBuilt',
 'BedroomAbvGr',
 '1stFlrSF',
 'GarageYrBlt',
 'OverallQual',
 'EnclosedPorch',
 'TotalBsmtSF',
 'MSSubClass',
 'LotArea',
 'BsmtHalfBath',
 'GarageArea',
 'OverallCond',
 'BsmtFinSF2',
 'OpenPorchSF',
 'ScreenPorch',
 'GarageCars',
 'GrLivArea',
 'KitchenAbvGr']

In [ ]:
nominal=['MSZoning','LandContour','Neighborhood']
ordinal=list(set(categorical_features)-set(nominal))

In [ ]:
target=['SalePrice']

In [ ]:
dataset_nominal=pd.get_dummies(dataset[nominal])

In [ ]:
for feature in ordinal:
  dataset[feature]=dataset[feature].astype('category').cat.codes

dataset_ordinal=dataset[ordinal]

In [ ]:
dataset[numerical_features]

,HalfBath,Fireplaces,WoodDeckSF,2ndFlrSF,YearRemodAdd,LotFrontage,PoolArea,MiscVal,BsmtFinSF1,MoSold,...,LotArea,BsmtHalfBath,GarageArea,OverallCond,BsmtFinSF2,OpenPorchSF,ScreenPorch,GarageCars,GrLivArea,KitchenAbvGr
0,1,0,0,854,2003,65.0,0,0,706,2,...,8450,0,548,5,0,61,0,2,1710,1
1,0,1,298,0,1976,80.0,0,0,978,5,...,9600,1,460,8,0,0,0,2,1262,1
2,1,1,0,866,2002,68.0,0,0,486,9,...,11250,0,608,5,0,42,0,2,1786,1
3,0,1,0,756,1970,60.0,0,0,216,2,...,9550,0,642,5,0,35,0,3,1717,1
4,1,1,192,1053,2000,84.0,0,0,655,12,...,14260,0,836,5,0,84,0,3,2198,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1,1,0,694,2000,62.0,0,0,0,8,...,7917,0,460,5,0,40,0,2,1647,1
1456,0,2,349,0,1988,85.0,0,0,790,2,...,13175,0,500,6,163,0,0,2,2073,1
1457,0,2,0,1152,2006,66.0,0,2500,275,5,...,9042,0,252,9,0,60,0,1,2340,1
1458,0,0,366,0,1996,68.0,0,0,49,4,...,9717,0,240,6,1029,0,0,1,1078,1


In [ ]:
new_dataset=pd.concat([dataset_nominal,dataset_ordinal,dataset[numerical_features]],axis=1)

In [ ]:
new_dataset.shape

(1460, 107)

# Standardization

In [ ]:
new_dataset.drop('SalePrice',axis=1,inplace=True)

In [ ]:
X=new_dataset.to_numpy()

In [ ]:
X[0]

array([0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 4.000e+00, 8.000e+00,
       0.000e+00, 0.000e+00, 2.000e+00, 1.000e+00, 4.000e+00, 1.000e+00,
       3.000e+00, 2.000e+00, 2.000e+00, 5.000e+00, 3.000e+00, 1.000e+00,
       0.000e+00, 4.000e+00, 4.000e+00, 2.000e+00, 1.000e+00, 2.000e+00,
       1.300e+01, 2.000e+00, 4.000e+00, 1.000e+00, 1.200e+01, 2.000e+00,
       4.000e+00, 2.000e+00, 1.000e+00, 5.000e+00, 4.000e+00, 0.000e+00,
       1.000e+00, 3.000e+00, 6.000e+00, 1.000e+00, 1.000e+00, 0.000e+00,
       0.000e+00, 8.540e+02, 2.003e+03, 6.500e+01, 0.000e+00, 0.000e+00,
       7.060e+02, 2.000e+00, 2.000e+00, 0.000e+00, 

In [ ]:
X=StandardScaler().fit_transform(X)

In [ ]:
X[0]

array([-0.08304548, -0.21585871, -0.10526316,  0.51813339, -0.41895507,
       -0.21235968, -0.18831089, -0.15899968,  0.33712564, -0.10854037,
       -0.03703704, -0.10526316, -0.20339487, -0.1398323 ,  2.95522137,
       -0.19025216, -0.27116307, -0.23917551, -0.16124951, -0.10854037,
       -0.1863522 , -0.42683279, -0.07875671, -0.22941573, -0.16998114,
       -0.23595776, -0.28963792, -0.13199092, -0.23106504, -0.20521398,
       -0.25018188, -0.13199092, -0.16347148, -0.08712888,  0.18797343,
        0.31386709, -0.02618016, -0.22571613, -0.03628912, -0.23012211,
        0.22403433, -0.49151573,  0.64046232, -0.40779461, -0.40936914,
        0.30941909,  0.75073056, -0.12579688, -0.88465767,  0.60466978,
        1.11993319,  0.28974476, -0.71775144, -0.03174026,  0.75162623,
       -0.29055229,  0.2085023 ,  0.06423821,  0.74309239, -0.77797579,
        0.36420746,  0.83559903, -0.12304604,  1.02668924,  0.303692  ,
       -0.41169079,  0.26381257,  0.28780036,  0.25597761, -1.23

In [ ]:
y=dataset[target]

In [ ]:
y=y.to_numpy()

In [ ]:
y.shape

(1460, 1)

In [ ]:
y

array([[208500],
       [181500],
       [223500],
       ...,
       [266500],
       [142125],
       [147500]])

In [ ]:
y=StandardScaler().fit_transform(y)

In [ ]:
y

array([[ 0.34727322],
       [ 0.00728832],
       [ 0.53615372],
       ...,
       [ 1.07761115],
       [-0.48852299],
       [-0.42084081]])

In [ ]:
X.shape

(1460, 106)

# Feature Extraction Using PCA

In [ ]:
pca=PCA(n_components=2)

In [ ]:
p_components=pca.fit_transform(X)

In [ ]:
p_components

array([[ 2.44330901, -1.27622346],
       [ 0.31646218,  2.63697438],
       [ 2.85722129, -0.80904028],
       ...,
       [ 1.2618266 , -0.86453546],
       [-2.85097326,  3.2925518 ],
       [-0.7773883 ,  2.78945542]])

In [ ]:
pca.explained_variance_ratio_

array([0.10560233, 0.0447209 ])

In [ ]:
pca_data = pd.DataFrame(p_components,columns=['PC1','PC2'])
target_data = pd.DataFrame(y,columns=['SalePrice'])
final_data = pd.concat([pca_data,target_data],axis=1)
final_data

,PC1,PC2,SalePrice
0,2.443309,-1.276223,0.347273
1,0.316462,2.636974,0.007288
2,2.857221,-0.809040,0.536154
3,-1.333156,0.423931,-0.515281
4,5.220986,-0.246901,0.869843
...,...,...,...
1455,1.335075,-2.288175,-0.074560
1456,1.112795,3.258780,0.366161
1457,1.261827,-0.864535,1.077611
1458,-2.850973,3.292552,-0.488523
